In [ ]:
IMAGE_NAME='sagemaker-slim-buster'
TAG = ':python39'

%cd container/3-1
!docker build -t {IMAGE_NAME}{TAG} . 
%cd ../../

In [ ]:
# boto3の機能を使ってリポジトリ名に必要な情報を取得する
import boto3
MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
REGION = boto3.session.Session().region_name
# ECR へ Docker Image の push
MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'

MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)
 
# # リポジトリの作成
!aws ecr delete-repository --repository-name {IMAGE_NAME} --force # 同名のリポジトリがあった場合削除
!aws ecr create-repository --repository-name {IMAGE_NAME}
 
!docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
!docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

In [ ]:
from sagemaker.estimator import Estimator

estimator = Estimator(
    image_uri = MY_IMAGE_URI,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.xlarge',
)
estimator.fit()

In [ ]:
IMAGE_NAME='sagemaker-slim-buster'
TAG = ':python39-sagemaker-training'

%cd container/3-2
!docker build -t {IMAGE_NAME}{TAG} . 
%cd ../../

# boto3の機能を使ってリポジトリ名に必要な情報を取得する
import boto3
MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
REGION = boto3.session.Session().region_name
# ECR へ Docker Image の push
MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'

MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)

!docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
!docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

In [ ]:
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-5/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-5')
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-5/concat.py',
    image_uri=MY_IMAGE_URI,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

In [ ]:
import tarfile
prefix = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts'].replace(f's3://{sagemaker.session.Session().default_bucket()}/','')
sagemaker.session.Session().download_data('./', sagemaker.session.Session().default_bucket(), key_prefix=prefix)
with tarfile.open('./model.tar.gz', 'r') as f:
    f.extractall()
!cat output.csv